In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'DBLP-Scholar'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[ 0.13545009, -0.07971626,  0.10099645,  0.08951191,  0.15098801,
         0.1405168 , -0.07481844, -0.06840061, -0.06755616, -0.09204526],
       [ 0.0177497 ,  0.13929001,  0.29482551, -0.03880866,  0.03188929,
         0.19253911,  0.06498195,  0.03459687,  0.13628159,  0.0490373 ],
       [-0.03739612, -0.20803324,  0.08919668,  0.08116343,  0.01080332,
         0.09584488, -0.18975069,  0.21440443, -0.02742382,  0.04598338],
       [ 0.15810355, -0.02757389, -0.18706606, -0.17377504,  0.02241619,
         0.1674271 ,  0.05137869, -0.06744694, -0.13667923,  0.00813331],
       [-0.01095836,  0.11097828, -0.18549512,  0.15501096, -0.08607292,
         0.09085475, -0.08408049, -0.16855947, -0.06814106, -0.03984858]]), array([[ 0.03017329, -0.02568807,  0.06055046, -0.2038736 , -0.11437309,
         0.16330275,  0.1412844 ,  0.00713558, -0.1420999 ,  0.11151886],
       [-0.05588311, -0.01691874,  0.21148423, -0.08613176,  0.05229428,
         0.06998206,  0.13201743, -0.06895

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


1
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(10):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.7700934579439253
K: 1, L: 2, Precision:0.9719626168224299
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.7560747663551401
K: 2, L: 2, Precision:0.9046728971962616
K: 2, L: 3, Precision:0.983177570093458
K: 2, L: 4, Precision:0.9990654205607477
K: 2, L: 5, Precision:0.9990654205607477
K: 2, L: 6, Precision:0.9990654205607477
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.5242990654205607
K: 3, L: 2, Precision:0.8074766355140187
K: 3, L: 3, Precision:0.9429906542056075
K: 3, L: 4, Precision:0.9728971962616823
K: 3, L: 5, Precision:0.9925233644859813
K: 3, L: 6, Precision:0.9981308411214953
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: